# Import the libraries

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

# Geometric Brownian Motion Class

In [ ]:
# Define the Geometric Brownian Motion (GBM) class
class GeometricBrownianMotion:
    def __init__(self, mu, sigma, S0):
        """
        Initialize the GBM process.

        Parameters:
        mu: Drift coefficient
        sigma: Volatility coefficient
        S0: Initial asset price
        """
        self.mu = mu
        self.sigma = sigma
        self.S0 = S0

    def generate_path(self, T, N):
        """
        Generate a single path of the GBM process.

        Parameters:
        T: Time horizon
        N: Number of time steps

        Returns:
        t: Array of time points
        S: Simulated asset prices
        """
        dt = T / N
        t = np.linspace(0, T, N)
        W = np.random.standard_normal(size=N)
        W = np.cumsum(W) * np.sqrt(dt)
        X = (self.mu - 0.5 * self.sigma**2) * t + self.sigma * W
        S = self.S0 * np.exp(X)
        return t, S


# Jump Diffusion Class

In [ ]:
# Define the Jump Diffusion class
class JumpDiffusion:
    def __init__(self, mu, sigma, S0, jump_mu, jump_sigma, jump_lambda):
        """
        Initialize the Jump Diffusion process.

        Parameters:
        mu: Drift coefficient
        sigma: Volatility coefficient
        S0: Initial asset price
        jump_mu: Mean of the jump size
        jump_sigma: Standard deviation of the jump size
        jump_lambda: Average number of jumps per unit time
        """
        self.mu = mu
        self.sigma = sigma
        self.S0 = S0
        self.jump_mu = jump_mu
        self.jump_sigma = jump_sigma
        self.jump_lambda = jump_lambda

    def generate_path(self, T, N):
        """
        Generate a single path of the Jump Diffusion process.

        Parameters:
        T: Time horizon
        N: Number of time steps

        Returns:
        t: Array of time points
        S: Simulated asset prices
        """
        dt = T / N
        t = np.linspace(0, T, N)
        W = np.random.standard_normal(size=N)
        W = np.cumsum(W) * np.sqrt(dt)
        X = (self.mu - 0.5 * self.sigma**2) * t + self.sigma * W

        # Jumps
        jumps = np.random.poisson(self.jump_lambda * dt, N) * \
                np.random.normal(self.jump_mu, self.jump_sigma, N)
        S = self.S0 * np.exp(X + jumps)
        return t, S


# Monte Carlo Simulation Class

In [ ]:
# Define the Monte Carlo Simulation class
class MonteCarloSimulation:
    def __init__(self, process, T, N, num_simulations):
        """
        Initialize the Monte Carlo simulation.

        Parameters:
        process: Stochastic process to simulate
        T: Time horizon
        N: Number of time steps
        num_simulations: Number of simulations to run
        """
        self.process = process
        self.T = T
        self.N = N
        self.num_simulations = num_simulations

    def run_simulation(self):
        """
        Run the Monte Carlo simulation.

        Returns:
        t: Array of time points
        simulations: Array of simulated asset prices
        """
        simulations = np.zeros((self.num_simulations, self.N))
        for i in range(self.num_simulations):
            t, S = self.process.generate_path(self.T, self.N)
            simulations[i, :] = S
        return t, simulations

    def estimate_distribution(self):
        """
        Estimate the distribution of the final prices.

        Returns:
        final_prices: Array of final prices from each simulation
        """
        _, simulations = self.run_simulation()
        final_prices = simulations[:, -1]
        return final_prices


# Plot

In [ ]:
# Function to plot the simulated paths
def plot_simulations(t, simulations, title):
    plt.figure(figsize=(10, 6))
    plt.plot(t, simulations.T, alpha=0.1)
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.show()

# Function to plot the distribution of final prices
def plot_final_distribution(final_prices, title):
    plt.figure(figsize=(10, 6))
    plt.hist(final_prices, bins=50, density=True)
    plt.title(title)
    plt.xlabel('Price')
    plt.ylabel('Frequency')
    plt.show()


# Interactive Simulation Function

In [ ]:
# Interactive function to run and plot Monte Carlo simulations
def interactive_simulation(model='GBM', mu=0.1, sigma=0.2, S0=100,
                           jump_mu=-0.2, jump_sigma=0.1, jump_lambda=1,
                           T=1, N=100, num_simulations=1000):
    if model == 'GBM':
        process = GeometricBrownianMotion(mu, sigma, S0)
        title = 'Geometric Brownian Motion Monte Carlo Simulations'
        dist_title = 'Distribution of Final Prices (GBM)'
    elif model == 'Jump Diffusion':
        process = JumpDiffusion(mu, sigma, S0, jump_mu, jump_sigma, jump_lambda)
        title = 'Jump Diffusion Monte Carlo Simulations'
        dist_title = 'Distribution of Final Prices (Jump Diffusion)'

    mc = MonteCarloSimulation(process, T, N, num_simulations)
    t, simulations = mc.run_simulation()
    final_prices = mc.estimate_distribution()

    plot_simulations(t, simulations, title)
    plot_final_distribution(final_prices, dist_title)


# Interactive widgets to modify parameters

In [ ]:
# Create interactive sliders and dropdowns
interact(
    interactive_simulation,
    model=['GBM', 'Jump Diffusion'],
    mu=FloatSlider(value=0.1, min=-0.5, max=0.5, step=0.01, description='Drift (mu)'),
    sigma=FloatSlider(value=0.2, min=0.0, max=1.0, step=0.01, description='Volatility (sigma)'),
    S0=FloatSlider(value=100, min=0, max=500, step=10, description='Initial Price (S0)'),
    jump_mu=FloatSlider(value=-0.2, min=-1.0, max=1.0, step=0.01, description='Jump Mean (jump_mu)'),
    jump_sigma=FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, description='Jump Std (jump_sigma)'),
    jump_lambda=FloatSlider(value=1, min=0, max=10, step=0.1, description='Jump Intensity (jump_lambda)'),
    T=FloatSlider(value=1, min=0.1, max=5.0, step=0.1, description='Time Horizon (T)'),
    N=IntSlider(value=100, min=10, max=500, step=10, description='Time Steps (N)'),
    num_simulations=IntSlider(value=1000, min=100, max=5000, step=100, description='Num Simulations')
);


interactive(children=(Dropdown(description='model', options=('GBM', 'Jump Diffusion'), value='GBM'), FloatSlid…